In [2]:
pip install fair-esm pandas scikit-learn torch lazypredict


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Peptide

ESM Feature Extraction**

In [7]:
import pandas as pd
import torch
import esm
import numpy as np

# Load the CSV file
file_path = '/kaggle/input/combineddb/combined_peptides.csv'
data = pd.read_csv(file_path)

# Load the pretrained ESM model
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()  # Adjust model as needed
batch_converter = alphabet.get_batch_converter()

# Move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Prepare the sequence data
sequences = data['sequence'].tolist()
labels = data['label'].tolist()

batch_size = 100  # Adjust the batch size based on your memory capacity
all_embeddings = []

for i in range(0, len(sequences), batch_size):
    batch_sequences = sequences[i:i+batch_size]
    batch_labels = labels[i:i+batch_size]
    
    data_tuples = [(f"sequence_{j}", seq) for j, seq in enumerate(batch_sequences)]
    _, _, batch_tokens = batch_converter(data_tuples)
    
    # Move batch tokens to GPU
    batch_tokens = batch_tokens.to(device)
    
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33])
    token_embeddings = results["representations"][33]
    
    # Move embeddings to CPU for further processing
    token_embeddings = token_embeddings.cpu()

    # Mean pooling to get fixed-size feature vectors
    def mean_pooling(token_embeddings, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    attention_mask = batch_tokens.ne(alphabet.padding_idx)
    attention_mask = attention_mask.cpu()  # Ensure attention mask is also on CPU
    sequence_embeddings = mean_pooling(token_embeddings, attention_mask)
    
    # Move sequence embeddings to CPU before converting to numpy
    all_embeddings.append(sequence_embeddings.cpu().numpy())

# Concatenate all the embeddings
all_embeddings = np.concatenate(all_embeddings, axis=0)

# Convert the embeddings to a DataFrame
embedding_df = pd.DataFrame(all_embeddings)
embedding_df['label'] = labels

# Save the DataFrame to a CSV file
embedding_df.to_csv('/kaggle/working/embeddings.csv', index=False)





In [8]:
from sklearn.model_selection import train_test_split

# Load the embeddings
embedding_df = pd.read_csv('/kaggle/working/embeddings.csv')

# Split the data into features and labels
X = embedding_df.drop(columns=['label'])
y = embedding_df['label']

# Split the data into training (60%) and testing (40%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Save the split datasets
X_train.to_csv('/kaggle/working/X_train.csv', index=False)
X_test.to_csv('/kaggle/working/X_test.csv', index=False)
y_train.to_csv('/kaggle/working/y_train.csv', index=False)
y_test.to_csv('/kaggle/working/y_test.csv', index=False)
X_val.to_csv('/kaggle/working/X_val.csv', index=False)
y_val.to_csv('/kaggle/working/y_val.csv', index=False)



In [9]:
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score

# Load the split datasets
X_train = pd.read_csv('/kaggle/working/X_train.csv')
X_test = pd.read_csv('/kaggle/working/X_test.csv')
X_val = pd.read_csv('/kaggle/working/X_val.csv')
y_train = pd.read_csv('/kaggle/working/y_train.csv').values.ravel()
y_test = pd.read_csv('/kaggle/working/y_test.csv').values.ravel()
y_val = pd.read_csv('/kaggle/working/y_val.csv').values.ravel()

# Initialize LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Train and test the models
models, predictions = clf.fit(X_train, X_val, y_train, y_val)

# Display the results
print(models)


 97%|█████████▋| 28/29 [02:27<00:04,  4.23s/it]

[LightGBM] [Info] Number of positive: 1812, number of negative: 1560
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.052981 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 326400
[LightGBM] [Info] Number of data points in the train set: 3372, number of used features: 1280
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.537367 -> initscore=0.149745
[LightGBM] [Info] Start training from score 0.149745


100%|██████████| 29/29 [02:38<00:00,  5.46s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.93               0.93     0.93      0.93   
LGBMClassifier                     0.93               0.93     0.93      0.93   
ExtraTreesClassifier               0.93               0.93     0.93      0.93   
SVC                                0.93               0.93     0.93      0.93   
RandomForestClassifier             0.93               0.93     0.93      0.93   
RidgeClassifierCV                  0.92               0.92     0.92      0.92   
LinearDiscriminantAnalysis         0.91               0.91     0.91      0.91   
RidgeClassifier                    0.91               0.91     0.91      0.91   
CalibratedClassifierCV             0.91               0.91     0.91      0.91   
PassiveAggressiveClassifier        0.91               0.91     0.91      0.91   
BaggingClassifier           

In [10]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the model dictionary including the specified models
model_dict = {
    'XGBClassifier': XGBClassifier(),
    'LGBMClassifier': LGBMClassifier(),
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'SVC': SVC(),
    'RandomForestClassifier': RandomForestClassifier(),
}

# Fit and evaluate each model
best_model_name = None
best_accuracy = 0

for model_name, model in model_dict.items():
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    test_predictions = model.predict(X_test)
    val_accuracy = accuracy_score(y_val, val_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    print(f"{model_name} validation accuracy: {val_accuracy:.4f}, test accuracy: {test_accuracy:.4f}")

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model_name = model_name

print(f"\nBest Model Name: {best_model_name}")

XGBClassifier validation accuracy: 0.9306, test accuracy: 0.9253
[LightGBM] [Info] Number of positive: 1812, number of negative: 1560
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055315 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 326400
[LightGBM] [Info] Number of data points in the train set: 3372, number of used features: 1280
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.537367 -> initscore=0.149745
[LightGBM] [Info] Start training from score 0.149745
LGBMClassifier validation accuracy: 0.9297, test accuracy: 0.9289
ExtraTreesClassifier validation accuracy: 0.9297, test accuracy: 0.9316
SVC validation accuracy: 0.8496, test accuracy: 0.8524
RandomForestClassifier validation accuracy: 0.9253, test accuracy: 0.9227

Best Model Name: XGBClassifier


# # Protein

In [3]:
import pandas as pd
import torch
import esm
import numpy as np
import gc
from torch.cuda.amp import autocast

# Load the CSV file
file_path = '/kaggle/input/combineddb/combined_protein.csv'
data = pd.read_csv(file_path)

# Load the pretrained ESM model
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()

# Move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Prepare the sequence data
sequences = data['sequence'].tolist()
labels = data['label'].tolist()

batch_size = 10  # Reduce the batch size to fit within the available GPU memory
all_embeddings = []

def mean_pooling(token_embeddings, attention_mask):
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

for i in range(0, len(sequences), batch_size):
    print(f"Processing batch {i//batch_size + 1}/{len(sequences)//batch_size + 1}")
    batch_sequences = sequences[i:i+batch_size]
    batch_labels = labels[i:i+batch_size]

    data_tuples = [(f"sequence_{j}", seq) for j, seq in enumerate(batch_sequences)]
    _, _, batch_tokens = batch_converter(data_tuples)
    print("Batch tokens:", batch_tokens.shape)

    # Move batch tokens to GPU
    batch_tokens = batch_tokens.to(device)

    with torch.no_grad():
        with autocast():
            results = model(batch_tokens, repr_layers=[33])
    token_embeddings = results["representations"][33]

    # Mean pooling to get fixed-size feature vectors
    attention_mask = batch_tokens.ne(alphabet.padding_idx)
    sequence_embeddings = mean_pooling(token_embeddings, attention_mask)

    # Move sequence embeddings to CPU before converting to numpy
    all_embeddings.append(sequence_embeddings.cpu().numpy())

    # Clear the cache
    torch.cuda.empty_cache()
    gc.collect()  # Invoke garbage collector to free up memory

# Concatenate all the embeddings
all_embeddings = np.concatenate(all_embeddings, axis=0)

# Convert the embeddings to a DataFrame
embedding_df = pd.DataFrame(all_embeddings)
embedding_df['label'] = labels

# Save the DataFrame to a CSV file
embedding_df.to_csv('/kaggle/working/embeddings_protien.csv', index=False)


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt" to /root/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm1b_t33_650M_UR50S-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S-contact-regression.pt


Processing batch 1/906
Batch tokens: torch.Size([10, 845])
Processing batch 2/906
Batch tokens: torch.Size([10, 563])
Processing batch 3/906
Batch tokens: torch.Size([10, 1004])
Processing batch 4/906
Batch tokens: torch.Size([10, 367])
Processing batch 5/906
Batch tokens: torch.Size([10, 918])
Processing batch 6/906
Batch tokens: torch.Size([10, 762])
Processing batch 7/906
Batch tokens: torch.Size([10, 513])
Processing batch 8/906
Batch tokens: torch.Size([10, 455])
Processing batch 9/906
Batch tokens: torch.Size([10, 741])
Processing batch 10/906
Batch tokens: torch.Size([10, 691])
Processing batch 11/906
Batch tokens: torch.Size([10, 711])
Processing batch 12/906
Batch tokens: torch.Size([10, 657])
Processing batch 13/906
Batch tokens: torch.Size([10, 798])
Processing batch 14/906
Batch tokens: torch.Size([10, 399])
Processing batch 15/906
Batch tokens: torch.Size([10, 790])
Processing batch 16/906
Batch tokens: torch.Size([10, 815])
Processing batch 17/906
Batch tokens: torch.Size

In [4]:
from sklearn.model_selection import train_test_split

# Load the embeddings
embedding_df = pd.read_csv('/kaggle/working/embeddings_protien.csv')

# Split the data into features and labels
X = embedding_df.drop(columns=['label'])
y = embedding_df['label']

# Split the data into training (60%) and testing (40%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Save the split datasets
X_train.to_csv('/kaggle/working/X_train_pro.csv', index=False)
X_test.to_csv('/kaggle/working/X_test_pro.csv', index=False)
y_train.to_csv('/kaggle/working/y_train_pro.csv', index=False)
y_test.to_csv('/kaggle/working/y_test_pro.csv', index=False)
X_val.to_csv('/kaggle/working/X_val_pro.csv', index=False)
y_val.to_csv('/kaggle/working/y_val_pro.csv', index=False)



In [5]:
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score

# Load the split datasets
X_train = pd.read_csv('/kaggle/working/X_train_pro.csv')
X_test = pd.read_csv('/kaggle/working/X_test_pro.csv')
X_val = pd.read_csv('/kaggle/working/X_val_pro.csv')
y_train = pd.read_csv('/kaggle/working/y_train_pro.csv').values.ravel()
y_test = pd.read_csv('/kaggle/working/y_test_pro.csv').values.ravel()
y_val = pd.read_csv('/kaggle/working/y_val_pro.csv').values.ravel()

# Initialize LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Train and test the models
models, predictions = clf.fit(X_train, X_val, y_train, y_val)

# Display the results
print(models)


 97%|█████████▋| 28/29 [03:43<00:04,  4.79s/it]

[LightGBM] [Info] Number of positive: 2053, number of negative: 3380
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.082112 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 326400
[LightGBM] [Info] Number of data points in the train set: 5433, number of used features: 1280
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.377876 -> initscore=-0.498574
[LightGBM] [Info] Start training from score -0.498574


100%|██████████| 29/29 [03:57<00:00,  8.18s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.97               0.97     0.97      0.97   
LogisticRegression                 0.97               0.97     0.97      0.97   
SVC                                0.97               0.97     0.97      0.97   
SGDClassifier                      0.97               0.96     0.96      0.97   
KNeighborsClassifier               0.96               0.96     0.96      0.96   
LGBMClassifier                     0.97               0.96     0.96      0.97   
RidgeClassifierCV                  0.96               0.96     0.96      0.96   
LinearDiscriminantAnalysis         0.96               0.96     0.96      0.96   
RidgeClassifier                    0.96               0.96     0.96      0.96   
ExtraTreesClassifier               0.97               0.96     0.96      0.97   
PassiveAggressiveClassifier 

In [6]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the model dictionary including the specified models
model_dict = {
    'XGBClassifier': XGBClassifier(),
    'LGBMClassifier': LGBMClassifier(),
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'SVC': SVC(),
    'RandomForestClassifier': RandomForestClassifier(),
}

# Fit and evaluate each model
best_model_name = None
best_accuracy = 0

for model_name, model in model_dict.items():
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    test_predictions = model.predict(X_test)
    val_accuracy = accuracy_score(y_val, val_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    print(f"{model_name} validation accuracy: {val_accuracy:.4f}, test accuracy: {test_accuracy:.4f}")

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model_name = model_name

print(f"\nBest Model Name: {best_model_name}")

XGBClassifier validation accuracy: 0.9691, test accuracy: 0.9763
[LightGBM] [Info] Number of positive: 2053, number of negative: 3380
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.089576 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 326400
[LightGBM] [Info] Number of data points in the train set: 5433, number of used features: 1280
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.377876 -> initscore=-0.498574
[LightGBM] [Info] Start training from score -0.498574
LGBMClassifier validation accuracy: 0.9647, test accuracy: 0.9768
ExtraTreesClassifier validation accuracy: 0.9647, test accuracy: 0.9757
SVC validation accuracy: 0.9431, test accuracy: 0.9432
RandomForestClassifier validation accuracy: 0.9630, test accuracy: 0.9735

Best Model Name: XGBClassifier
